In [1]:
from langchain_community.llms import GooglePalm

In [4]:
import configparser

config = configparser.RawConfigParser()
config.read('../config.config')
googleapi_key = config.get('Keys', 'googleapi_key')
llm = GooglePalm(google_api_key = googleapi_key, temperature = 0.7 )

In [5]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [13]:
db_user = "root"
db_password = "1234"
db_host = "localhost"
db_name = "vickys_auto_hub"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	vehicle_id INTEGER(11) NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(vehicle_id) REFERENCES vehicles (vehicle_id)
)DEFAULT CHARSET=utf8 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	vehicle_id	pct_discount
1	29	22.00
2	88	22.00
3	41	28.00
*/


CREATE TABLE vehicles (
	vehicle_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	make VARCHAR(50) NOT NULL, 
	model VARCHAR(50) NOT NULL, 
	year YEAR(4) NOT NULL, 
	color VARCHAR(50) NOT NULL, 
	body_style ENUM('Sedan','SUV','Truck','Van','Coupe','Convertible') NOT NULL, 
	kms_driven INTEGER(11) NOT NULL, 
	price DECIMAL(10, 2) NOT NULL, 
	stock_quantity INTEGER(11) NOT NULL, 
	PRIMARY KEY (vehicle_id)
)DEFAULT CHARSET=utf8 ENGINE=InnoDB

/*
3 rows from vehicles table:
vehicle_id	make	model	year	color	body_style	kms_driven	price	stock_quantity
1	Kia	F-150	2016	White	Coupe	14213	19341.00	0
2	Volkswag

In [81]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain.run("How many Volkswagen are available beyond the year 2010?")



> Entering new SQLDatabaseChain chain...
How many Volkswagen are available beyond the year 2010?
SQLQuery:SELECT count(*) FROM vehicles WHERE make = 'Volkswagen' AND year > 2010
SQLResult: [(9,)]
Answer:9
> Finished chain.


In [66]:
qns2 = db_chain("How much is the total price for all quantities of Kia F-150 models?")



> Entering new SQLDatabaseChain chain...
How much is the total price for all quantities of Kia F-150 models?
SQLQuery:SELECT sum(price) FROM vehicles WHERE model = 'F-150' AND make = 'Kia'
SQLResult: [(Decimal('37973.00'),)]
Answer:37973.00
> Finished chain.


In [82]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM vehicles WHERE  model = 'F-150' AND make = 'Kia'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM vehicles WHERE  model = 'F-150' AND make = 'Kia'
SQLQuery:SELECT SUM(price*stock_quantity) FROM vehicles WHERE  model = 'F-150' AND make = 'Kia'
SQLResult: [(Decimal('261376.00'),)]
Answer:261376.00
> Finished chain.


In [53]:
qns3 = db_chain("If all the available quantities of Nissan cars are sold with discounts applied, how much revenue will we make? ")



> Entering new SQLDatabaseChain chain...
If all the available quantities of Nissan cars are sold with discounts applied, how much revenue will we make? 
SQLQuery:SELECT SUM(v.price - (v.price * d.pct_discount)) 
FROM vehicles AS v JOIN discounts AS d ON v.vehicle_id = d.vehicle_id 
WHERE v.make = 'Nissan' AND v.stock_quantity > 0
SQLResult: [(Decimal('-109094.0000'),)]
Answer:-109094.0000
> Finished chain.


In [51]:
qns3

'2257611.68'

In [83]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, vehicle_id from vehicles where make = 'Nissan'
group by vehicle_id) a left join discounts on a.vehicle_id = discounts.vehicle_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, vehicle_id from vehicles where make = 'Nissan'
group by vehicle_id) a left join discounts on a.vehicle_id = discounts.vehicle_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, vehicle_id from vehicles where make = 'Nissan'
group by vehicle_id) a left join discounts on a.vehicle_id = discounts.vehicle_id
SQLResult: [(Decimal('2257611.68000000'),)]
Answer:2257611.68
> Finished chain.


In [55]:
qns3

'2257611.68'

In [84]:
qns4 = db_chain.run("select sum(price * stock_quantity from vehicles_ where make = 'Kia'")



> Entering new SQLDatabaseChain chain...
select sum(price * stock_quantity from vehicles_ where make = 'Kia'
SQLQuery:SELECT SUM(price * stock_quantity) FROM vehicles WHERE make = 'Kia'
SQLResult: [(Decimal('1583562.00'),)]
Answer:1583562.00
> Finished chain.


In [31]:
qns5 = db_chain.run("How many SUVs from Chevrolet do I actually have?")



> Entering new SQLDatabaseChain chain...
How many SUVs from Chevrolet do I actually have?
SQLQuery:SELECT count(*) FROM vehicles WHERE make = 'Chevrolet' AND body_style = 'SUV'
SQLResult: [(3,)]
Answer:3
> Finished chain.


In [85]:
qns5 = db_chain.run("select sum(stock_quantity) FROM vehicles AS v where v.make = 'Chevrolet' AND v.body_style = 'SUV';")



> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) FROM vehicles AS v where v.make = 'Chevrolet' AND v.body_style = 'SUV';
SQLQuery:SELECT sum(stock_quantity) FROM vehicles AS v WHERE v.make = 'Chevrolet' AND v.body_style = 'SUV'
SQLResult: [(Decimal('12'),)]
Answer:12
> Finished chain.


In [101]:
qns6 = db_chain.run("How many Chevrolet cars do I actually have?")



> Entering new SQLDatabaseChain chain...
How many Chevrolet cars do I actually have?
SQLQuery:SELECT COUNT(make) FROM vehicles WHERE make = 'Chevrolet'
SQLResult: [(12,)]
Answer:12
> Finished chain.


In [102]:
qns6 = db_chain.run("Select sum(stock_quantity) from vehicles where make= 'Chevrolet'")



> Entering new SQLDatabaseChain chain...
Select sum(stock_quantity) from vehicles where make= 'Chevrolet'
SQLQuery:SELECT SUM(stock_quantity) FROM vehicles WHERE make = 'Chevrolet'
SQLResult: [(Decimal('57'),)]
Answer:57
> Finished chain.


### Few shot Learning

In [103]:
few_shots = [
    {'Question' : "How many Volkswagen are available beyond the year 2010?",
     'SQLQuery' : "SELECT count(*) FROM vehicles WHERE make = 'Volkswagen' AND year > 2010",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price for all quantities of Kia F-150 models?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM vehicles WHERE  model = 'F-150' AND make = 'Kia'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If all the available quantities of Nissan cars are sold with discounts applied, how much revenue will we make?" ,
     'SQLQuery' : "select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, vehicle_id from vehicles where make = 'Nissan' group by vehicle_id) a left join discounts on a.vehicle_id = discounts.vehicle_id",
     'SQLResult': "Result of the SQL query",
     'Answer': qns3},
     {'Question' : "If we have to sell all the Kia vehicles today. How much revenue will be generated without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM vehicles WHERE make = 'Kia'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many SUVs from Chevrolet do I actually have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM vehicles AS v where v.make = 'Chevrolet' AND v.body_style = 'SUV'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5},
    {'Question': "How many Chevrolet cars do I actually have?",
     'SQLQuery' : "Select sum(stock_quantity) from vehicles where make= 'Chevrolet'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns6}
    
]

In [118]:
few_shots

[{'Question': 'How many Volkswagen are available beyond the year 2010?',
  'SQLQuery': "SELECT count(*) FROM vehicles WHERE make = 'Volkswagen' AND year > 2010",
  'SQLResult': 'Result of the SQL query',
  'Answer': '9'},
 {'Question': 'How much is the total price for all quantities of Kia F-150 models?',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM vehicles WHERE  model = 'F-150' AND make = 'Kia'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '261376.00'},
 {'Question': 'If all the available quantities of Nissan cars are sold with discounts applied, how much revenue will we make?',
  'SQLQuery': "select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, vehicle_id from vehicles where make = 'Nissan' group by vehicle_id) a left join discounts on a.vehicle_id = discounts.vehicle_id",
  'SQLResult': 'Result of the SQL query',
  'Answer': '2257611.68'},
 {'Question': 'If we have to 

In [104]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [105]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [106]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2)

In [107]:
example_selector.select_examples({"Question":"How many sedans I have left in my store?"})

[{'Answer': '12',
  'Question': 'How many SUVs from Chevrolet do I actually have?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM vehicles AS v where v.make = 'Chevrolet' AND v.body_style = 'SUV'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '12',
  'Question': 'How many SUVs from Chevrolet do I actually have?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM vehicles AS v where v.make = 'Chevrolet' AND v.body_style = 'SUV'",
  'SQLResult': 'Result of the SQL query'}]

In [108]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [109]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [110]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [111]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [112]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)


In [113]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt = few_shot_prompt)

In [115]:
new_chain("What is the total quanity of Toyota cars left in store?")



> Entering new SQLDatabaseChain chain...
What is the total quanity of Toyota cars left in store?
SQLQuery:select sum(stock_quantity) as total_quantity from vehicles where make = 'Toyota'
SQLResult: [(Decimal('52'),)]
Answer:52
> Finished chain.


{'query': 'What is the total quanity of Toyota cars left in store?',
 'result': '52'}

In [116]:
new_chain("How many models of Toyota cars left in store?")



> Entering new SQLDatabaseChain chain...
How many models of Toyota cars left in store?
SQLQuery:SELECT count(distinct(model)) from vehicles where make = 'Toyota'
SQLResult: [(5,)]
Answer:5
> Finished chain.


{'query': 'How many models of Toyota cars left in store?', 'result': '5'}

In [117]:
new_chain("WHich make generates highest revenue without discounts?")



> Entering new SQLDatabaseChain chain...
WHich make generates highest revenue without discounts?
SQLQuery:select make, sum(price*stock_quantity) as total_revenue from vehicles where price*stock_quantity > (select max(price*stock_quantity) from vehicles where price*stock_quantity < (select sum(price*stock_quantity) from vehicles)) group by make order by total_revenue desc limit 1
SQLResult: 
Answer:Kia 48159.00
> Finished chain.


{'query': 'WHich make generates highest revenue without discounts?',
 'result': 'Kia 48159.00'}